In [5]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc

# Sample data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Create a linear regression model
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Predict using the model
X_test = np.array([[3, 5], [4, 6]])
y_pred = model.predict(X_test)

print(y_pred)

[16. 19.]


Prendre en compte le nombre de risque pris en compte ? 

In [9]:
ds = xr.open_dataset('results/results_run_2031.nc')
ds

<xarray.Dataset> Size: 725kB
Dimensions:                                                                (
                                                                            A MATRIX YEARS I: 15,
                                                                            AEZ I: 4,
                                                                            AGE ADULTS I: 10,
                                                                            AGE CHAIN I: 16,
                                                                            AGE CHAIN MIDDLE I: 15,
                                                                            ...
                                                                            VINTAGING ANNUAL: 10,
                                                                            bottom: 1,
                                                                            lower: 3,
                                                                            pprofile: 4,
                                                                            upper: 3,
                                                                            time: 27)
Coordinates: (12/254)
  * A MATRIX YEARS I                                                       (A MATRIX YEARS I) <U8 480B ...
  * AEZ I                                                                  (AEZ I) <U13 208B ...
  * AGE ADULTS I                                                           (AGE ADULTS I) <U6 240B ...
  * AGE CHAIN I                                                            (AGE CHAIN I) <U7 448B ...
  * AGE CHAIN MIDDLE I                                                     (AGE CHAIN MIDDLE I) <U6 360B ...
  * AGE CHAIN YOUNG I                                                      (AGE CHAIN YOUNG I) <U6 360B ...
    ...                                                                     ...
  * VINTAGING ANNUAL                                                       (VINTAGING ANNUAL) <U3 120B ...
  * bottom                                                                 (bottom) <U6 24B ...
  * lower                                                                  (lower) <U6 72B ...
  * pprofile                                                               (pprofile) <U9 144B ...
  * upper                                                                  (upper) <U6 72B ...
  * time                                                                   (time) float64 216B ...
Data variables: (12/162)
    define_46_eq_damage_function                                           (time) float64 216B ...
    dice_5_eq_damage_function                                              (time) float64 216B ...
    fund_a1_eq_total_agricultural_impact                                   (time, REGIONS 35 I) float64 8kB ...
    fund_a2_delayed_agricultural_impact_of_the_rate_of_climate_change      (time, REGIONS 35 I) float64 8kB ...
    fund_a2_eq_agricultural_impact_of_the_rate_of_climate_change           (time, REGIONS 35 I) float64 8kB ...
    fund_a2_rate_of_temperature_change                                     (time, REGIONS 35 I) float64 8kB ...
    ...                                                                     ...
    fund_w1_mu_parameter                                                   float64 8B ...
    fund_w1_tau_parameter                                                  float64 8B ...
    fund_extra_initial_revenue                                             (REGIONS 35 I) float64 280B ...
    fund_extra_initial_gdp                                                 (REGIONS 35 I) float64 280B ...
    fund_extra_initial_population                                          (REGIONS 35 I) float64 280B ...
    witness_epsilon                                                        float64 8B ...
Attributes:
    description:   Results for simulation run on Mon Jul  1 01:48:52 2024 usi...
    model_file:    WILIAM_v1.3\WILIAM.py
    timestep:   

In [16]:
df = ds['fund_tot_eq_fund_total_impact'].to_dataframe().reset_index()
df

,time,REGIONS 35 I,fund_tot_eq_fund_total_impact
0,2005.0,AUSTRIA,NaN
1,2005.0,BELGIUM,2.846619e+13
2,2005.0,BULGARIA,3.229255e+08
3,2005.0,CROATIA,2.996817e+08
4,2005.0,CYPRUS,1.631465e+12
...,...,...,...
940,2031.0,INDIA,2.005023e+10
941,2031.0,LATAM,3.258582e+10
942,2031.0,RUSSIA,1.703461e+10
943,2031.0,USMCA,1.315974e+11


In [23]:
import statsmodels.api as sm

# Create a dataframe with the variables for regression
df_regression = ds['fund_tot_eq_fund_total_impact'].to_dataframe().reset_index()
df_regression = df_regression.dropna()

# Create dummy variables for year and region
df_regression = pd.get_dummies(df_regression, columns=['time', 'REGIONS 35 I'], drop_first=True)

# Separate the dependent variable and independent variables
y = df_regression['fund_tot_eq_fund_total_impact']
X = df_regression.drop('fund_tot_eq_fund_total_impact', axis=1)


# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the regression model

X = X.astype(float)
y = y.astype(float)


model = sm.OLS(y, X)
results = model.fit()

# Print the regression results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     fund_tot_eq_fund_total_impact   R-squared:                       0.964
Model:                                       OLS   Adj. R-squared:                  0.961
Method:                            Least Squares   F-statistic:                     361.9
Date:                           Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                                   11:54:32   Log-Likelihood:                -24882.
No. Observations:                            783   AIC:                         4.987e+04
Df Residuals:                                728   BIC:                         5.013e+04
Df Model:                                     54                                         
Covariance Type:                       nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                        2.781e+13    4.2e+12      6.615      0.000    1.96e+13    3.61e+13
time_2006.0                  7.406e+11   4.17e+12      0.178      0.859   -7.44e+12    8.92e+12
time_2007.0                   1.56e+12   4.17e+12      0.374      0.708   -6.62e+12    9.74e+12
time_2008.0                  2.436e+12   4.17e+12      0.585      0.559   -5.74e+12    1.06e+13
time_2009.0                  3.359e+12   4.17e+12      0.806      0.420   -4.82e+12    1.15e+13
time_2010.0                  4.321e+12   4.17e+12      1.037      0.300   -3.86e+12    1.25e+13
time_2011.0                  4.993e+12   4.17e+12      1.199      0.231   -3.18e+12    1.32e+13
time_2012.0                  5.661e+12   4.17e+12      1.359      0.175   -2.52e+12    1.38e+13
time_2013.0                  6.421e+12   4.17e+12      1.541      0.124   -1.76e+12    1.46e+13
time_2014.0                  7.252e+12   4.17e+12      1.741      0.082   -9.27e+11    1.54e+13
time_2015.0                  8.142e+12   4.17e+12      1.955      0.051   -3.63e+10    1.63e+13
time_2016.0                  9.235e+12   4.17e+12      2.217      0.027    1.06e+12    1.74e+13
time_2017.0                  1.009e+13   4.17e+12      2.423      0.016    1.92e+12    1.83e+13
time_2018.0                  1.127e+13   4.17e+12      2.706      0.007    3.09e+12    1.95e+13
time_2019.0                  1.263e+13   4.17e+12      3.033      0.003    4.45e+12    2.08e+13
time_2020.0                  1.398e+13   4.17e+12      3.355      0.001     5.8e+12    2.22e+13
time_2021.0                  1.516e+13   4.17e+12      3.639      0.000    6.98e+12    2.33e+13
time_2022.0                  1.616e+13   4.17e+12      3.880      0.000    7.99e+12    2.43e+13
time_2023.0                  1.696e+13   4.17e+12      4.070      0.000    8.78e+12    2.51e+13
time_2024.0                  1.758e+13   4.17e+12      4.220      0.000     9.4e+12    2.58e+13
time_2025.0                  1.791e+13   4.17e+12      4.300      0.000    9.74e+12    2.61e+13
time_2026.0                  1.811e+13   4.17e+12      4.349      0.000    9.94e+12    2.63e+13
time_2027.0                  1.802e+13   4.17e+12      4.325      0.000    9.84e+12    2.62e+13
time_2028.0                  1.764e+13   4.17e+12      4.235      0.000    9.46e+12    2.58e+13
time_2029.0                  1.721e+13   4.17e+12      4.131      0.000    9.03e+12    2.54e+13
time_2030.0                  1.652e+13   4.17e+12      3.965      0.000    8.34e+12    2.47e+13
time_2031.0                  1.573e+13   4.17e+12      3.776      0.000    7.55e+12    2.39e+13
REGIONS 35 I_BULGARIA       -3.852e+13   4.32e+12     -8.922      0.000    -4.7e+1